In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import string
from textblob import TextBlob
import csv
import numpy as np

In [2]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Shirish\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Shirish\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
df = pd.read_csv('Input.csv')
df

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...
...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...
110,147,https://insights.blackcoffer.com/the-future-of...
111,148,https://insights.blackcoffer.com/big-data-anal...
112,149,https://insights.blackcoffer.com/business-anal...


In [4]:
li = [url for url in df['URL']]
li

['https://insights.blackcoffer.com/ai-in-healthcare-to-improve-patient-outcomes/',
 'https://insights.blackcoffer.com/what-if-the-creation-is-taking-over-the-creator/',
 'https://insights.blackcoffer.com/what-jobs-will-robots-take-from-humans-in-the-future/',
 'https://insights.blackcoffer.com/will-machine-replace-the-human-in-the-future-of-work/',
 'https://insights.blackcoffer.com/will-ai-replace-us-or-work-with-us/',
 'https://insights.blackcoffer.com/man-and-machines-together-machines-are-more-diligent-than-humans-blackcoffe/',
 'https://insights.blackcoffer.com/in-future-or-in-upcoming-years-humans-and-machines-are-going-to-work-together-in-every-field-of-work/',
 'https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/',
 'https://insights.blackcoffer.com/how-machine-learning-will-affect-your-business/',
 'https://insights.blackcoffer.com/deep-learning-impact-on-areas-of-e-learning/',
 'https://insights.blackcoffer.com/how-to-protect-fu

In [5]:
articles = []
for url in li:
    r = requests.get(url)
    data = r.text
    soup = BeautifulSoup(data, "html.parser")
    articles.append(soup.get_text())

print(f'Total {len(articles)} articles saved')

Total 114 articles saved


In [6]:
for i in range(len(articles)):
    articles[i]= articles[i].replace('\n','')

In [7]:
stop_words = list(set(stopwords.words('english')))

In [8]:
sentences = []
for article in articles:
    sentences.append(len(sent_tokenize(article)))

In [9]:
len(sentences)

114

In [10]:
cleaned_articles = [' ']*len(articles)

In [11]:
for i in range(len(articles)):
    for w in stop_words:
        cleaned_articles[i]= articles[i].replace(' '+w+' ',' ').replace('?','').replace('.','').replace(',','').replace('!','')

In [12]:
len(cleaned_articles)

114

In [13]:
words = []
for article in articles:
    words.append(len(word_tokenize(article)))

In [27]:
len(words)

114

In [14]:
words_cleaned = []
for article in cleaned_articles:
    words_cleaned.append(len(word_tokenize(article)))

In [15]:
dictionary = pd.read_excel('MasterDictionary.xlsx')

In [16]:
positive_words = list(dictionary[dictionary['Positive']==2009]['Word'])
positive_words = [word.lower() for word in positive_words]

In [17]:
negative_words = list(dictionary[dictionary['Negative']==2009]['Word'])

In [18]:
positive_score = [0]*len(articles)
for i in range(len(articles)):
    for word in positive_words:
        for letter in cleaned_articles[i].lower().split(' '):
            if letter==word:
                positive_score[i]+=1

In [19]:
negative_score = [0]*len(articles)
for i in range(len(articles)):
    for word in negative_words:
        for letter in cleaned_articles[i].upper().split(' '):
            if letter==word:
                negative_score[i]+=1

In [20]:
len(positive_score)

114

In [21]:
len(negative_score)

114

In [22]:
words_cleaned = np.array(words_cleaned)
sentences = np.array(sentences)

In [23]:
df['POSITIVE SCORE'] = positive_score
df['NEGATIVE SCORE'] = negative_score

In [24]:
df['POLARITY SCORE'] = (df['POSITIVE SCORE']-df['NEGATIVE SCORE'])/ ((df['POSITIVE SCORE'] +df['NEGATIVE SCORE']) + 0.000001)

In [25]:
df['SUBJECTIVITY SCORE'] = (df['POSITIVE SCORE'] + df['NEGATIVE SCORE'])/( (words_cleaned) + 0.000001)

In [26]:
df['AVG SENTENCE LENGTH'] = np.array(words)/np.array(sentences)

In [27]:
complex_words = []
sylabble_counts = []

In [28]:
for article in articles:
    sylabble_count=0
    d=article.split()
    ans=0
    for word in d:
        count=0
        for i in range(len(word)):
            if(word[i]=='a' or word[i]=='e' or word[i] =='i' or word[i] == 'o' or word[i] == 'u'):
                count+=1
            if(i==len(word)-2 and (word[i]=='e' and word[i+1]=='d')):
                count-=1;
            if(i==len(word)-2 and (word[i]=='e' and word[i+1]=='s')):
                count-=1;
                sylabble_count+=count
                if(count>2):
                    ans+=1
    sylabble_counts.append(sylabble_count)
    complex_words.append(ans)    

In [29]:
len(complex_words)

114

In [30]:
df['PERCENTAGE OF COMPLEX WORDS'] = np.array(complex_words)/np.array(words)

In [31]:
df['FOG INDEX'] = 0.4 * (df['AVG SENTENCE LENGTH'] + df['PERCENTAGE OF COMPLEX WORDS'])

In [32]:
df['AVG NUMBER OF WORDS PER SENTENCES'] = df['AVG SENTENCE LENGTH']

In [33]:
df['COMPLEX WORD COUNT'] = complex_words

In [34]:
df['WORD COUNT'] = words

In [35]:
df['SYLLABLE PER WORD'] = np.array(sylabble_counts)/np.array(words)

In [46]:
total_characters = []
for article in articles:
    characters = 0
    for word in article.split():
        characters+=len(word)
    total_characters.append(characters)  

In [47]:
len(total_characters)

114

In [39]:
personal_nouns = []
personal_noun =['I', 'we','my', 'ours','and' 'us','My','We','Ours','Us','And'] 
for article in articles:
    ans=0
    for word in article:
        if word in personal_noun:
            ans+=1
    personal_nouns.append(ans)

In [48]:
len(personal_nouns)

114

In [40]:
df['PERSONAL PRONOUN'] = personal_nouns

In [49]:
df['AVG WORD LENGTH'] = np.array(total_characters)/np.array(words)

In [50]:
df

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCES,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUN,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,40,19,0.355932,0.022877,47.474576,0.011067,18.994257,47.474576,31,2801,0.071760,98,5.699036
1,38,https://insights.blackcoffer.com/what-if-the-c...,42,31,0.150685,0.032927,32.421053,0.009334,12.972155,32.421053,23,2464,0.076299,89,5.142045
2,39,https://insights.blackcoffer.com/what-jobs-wil...,41,38,0.037975,0.031752,36.743243,0.014711,14.703182,36.743243,40,2719,0.086061,80,5.531078
3,40,https://insights.blackcoffer.com/will-machine-...,47,19,0.424242,0.027363,30.964706,0.005319,12.388010,30.964706,14,2632,0.051672,123,5.201748
4,41,https://insights.blackcoffer.com/will-ai-repla...,42,24,0.272727,0.025731,36.363636,0.008571,14.548883,36.363636,24,2800,0.066429,84,5.253214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...,24,19,0.116279,0.025982,40.568182,0.010084,16.231306,40.568182,18,1785,0.066667,67,5.861625
110,147,https://insights.blackcoffer.com/the-future-of...,37,11,0.541667,0.020348,46.851852,0.010277,18.744851,46.851852,26,2530,0.076285,89,5.372727
111,148,https://insights.blackcoffer.com/big-data-anal...,26,45,-0.267606,0.036617,30.507246,0.009501,12.206699,30.507246,20,2105,0.064133,72,5.476010
112,149,https://insights.blackcoffer.com/business-anal...,32,7,0.641026,0.026549,58.481481,0.010766,23.396899,58.481481,17,1579,0.077897,59,6.127296


In [51]:
df.to_excel('Output Data Structure.xlsx')